In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from torchvision import transforms
from torch.utils import data
from datetime import datetime
import itertools

from dataset.db_query import *
from dataset.dataset import SingleSeasonSingleLeagueTimeSlices, SingleSeasonSingleLeague
from dataset.util import pretty_print_match_ts, pretty_print_match
from dataset.train_valid_test_loader import *

import logging
logging.basicConfig(level=logging.DEBUG)


In [3]:
sqlpath = "../data/database.sqlite"


In [4]:
def collect_data_stats(data):
    num_home_wins = 0
    num_away_wins = 0
    num_draws = 0

    for x, y in data:
        if y[0] == 1:
            num_home_wins += 1
        if y[1] == 1: 
            num_draws += 1
        if y[2] == 1:
            num_away_wins += 1
    
    N = len(data)
    assert num_away_wins + num_draws + num_home_wins == N
    
    return num_home_wins / N, num_draws / N, num_away_wins / N

In [5]:
data = make_small_train_set(sqlpath, use_ts=True, ts_size=50)

INFO:root:got 1458 rows from db
INFO:root:got 183978 rows from db
INFO:root:got 306 rows (league: NET (id 13274), season: 2014/2015) from db
DEBUG:root:time for loading sql data 21.348589420318604
DEBUG:root:time for processing data 13.187032461166382


In [7]:
data[221]

({'home_team_name': 'Excelsior',
  'away_team_name': 'SC Heerenveen',
  'team_home': tensor([53., 69., 48., 45., 43., 29., 41., 36.]),
  'team_away': tensor([53., 46., 65., 43., 46., 46., 44., 48.]),
  'players_home': [tensor([63., 68., 25., 25., 25., 30., 20., 25., 25., 25., 44., 21., 53., 48.,
           49., 58., 48., 44., 66., 31., 75., 25., 44., 23., 25., 25., 22., 25.,
           25., 25., 65., 58., 63., 65., 65.]),
   tensor([62., 66., 66., 50., 75., 62., 26., 57., 39., 27., 64., 59., 66., 48.,
           60., 55., 68., 45., 82., 65., 65., 48., 66., 58., 63., 51., 47., 64.,
           64., 63., 13.,  9., 10., 11.,  5.]),
   tensor([65., 68., 29., 22., 64., 58., 25., 38., 25., 45., 60., 52., 61., 66.,
           53., 55., 56., 60., 68., 66., 77., 43., 65., 68., 20., 39., 60., 67.,
           68., 63.,  7., 15.,  8., 12., 13.]),
   tensor([64., 69., 31., 20., 67., 56., 24., 42., 34., 29., 58., 59., 58., 63.,
           47., 60., 53., 37., 70., 69., 84., 25., 62., 66., 30., 34., 53

In [26]:
loader = torch.utils.data.dataloader.DataLoader(data, batch_size=3)

In [29]:
for i, (x, y) in enumerate(loader):
    if i == 2:
        print(x["home_team_name"])
        print(x["home_team_history"])
        break

['Go Ahead Eagles', 'Ajax', 'Willem II']
[]
